# Wavetable

A few experiments with the Wavetable crate.

First we need to do some setup to use Plotly for displaying graphs.

In [2]:
:dep rustfft = "5.0"
:dep plotly = { version = ">=0.6.0" }
:dep itertools-num = "0.1.3"

In [3]:
extern crate plotly;
extern crate rand_distr;
extern crate itertools_num;
extern crate itertools;

In [4]:
// Import 3rd party crates
// TODO: We probably don't need all of these, clean up
use itertools_num::linspace;
use plotly::{Bar, NamedColor, Plot, Rgb, Rgba, Scatter, Surface};
use plotly::common::{ColorScale, ColorScalePalette, DashType, Fill, Font, Line, LineShape, Marker, Mode, Side, Title};
use plotly::layout::{Axis, BarMode, GridPattern, Layout, LayoutGrid, Legend, RowOrder, TicksDirection};
use rand_distr::{Distribution, Normal, Uniform};
use rustfft::Fft;
use rustfft::FftPlanner;
use rustfft::num_complex::Complex;
use rustfft::num_traits::Zero;

Then we load the Wavetable crate and create some handler objects.

In [56]:
:dep wavetable = { path = "/Users/ingo/Documents/Programming/src/rust/wavetable/" }

use wavetable::{Wavetable, WavetableRef, WtManager, WtReader, Harmonic};

In [57]:
let mut wt_manager = WtManager::new(44100.0);
let basic_wave_id = 0;
wt_manager.add_basic_tables(basic_wave_id);

Define some helper functions

In [58]:
fn add_trace(plot: &mut Plot, data: &Vec<f32>, num_values: usize, index: usize) {
    let t: Vec<f64> = linspace(0., num_values as f64, num_values).collect();
    let trace = Scatter::new(t, data.clone())
        .x_axis(&format!("x{}", index)).y_axis(&format!("y{}", index));
    plot.add_trace(trace);
}

fn add_trace_complex(plot: &mut Plot, data: &Vec<Complex<f32>>, num_values: usize, index: usize) {
    let t: Vec<f64> = linspace(0., num_values as f64, num_values).collect();
    //let im: Vec<f32> = data.iter().map(|v| v.im).collect();
    //let re: Vec<f32> = data.iter().map(|v| v.re).collect();
    let mut im: Vec<f32> = vec!();
    let mut re: Vec<f32> = vec!();
    for v in data {
        im.push(v.im);
        re.push(v.re);
    }
    let trace = Scatter::new(t.clone(), im)
        .x_axis(&format!("x{}", index)).y_axis(&format!("y{}", index));
    plot.add_trace(trace);
    let trace = Scatter::new(t, re)
        .x_axis(&format!("x{}", index)).y_axis(&format!("y{}", index));
    plot.add_trace(trace);
}

/*
fn add_trace_harmonic(plot: &mut Plot, data: &Vec<Harmonic>, num_values: usize, index: usize) {
    let t: Vec<f64> = linspace(0., num_values as f64, num_values).collect();
    //let im: Vec<f32> = data.iter().map(|v| v.im).collect();
    //let re: Vec<f32> = data.iter().map(|v| v.re).collect();
    let mut mag: Vec<f32> = vec!();
    let mut phase: Vec<f32> = vec!();
    for v in data {
        mag.push(v.mag);
        phase.push(v.phase);
    }
    let trace = Scatter::new(t.clone(), mag)
        .x_axis(&format!("x{}", index)).y_axis(&format!("y{}", index));
    plot.add_trace(trace);
    let trace = Scatter::new(t, phase)
        .x_axis(&format!("x{}", index)).y_axis(&format!("y{}", index));
    plot.add_trace(trace);
}
*/

fn get_surface_plot(wt_ref: &WavetableRef, table_id: usize) -> Box<Surface<f64, f64, f32>> {
    let num_octaves = wt_ref.num_octaves;
    let num_values = wt_ref.num_values;
    let wt = wt_ref.get_wave(table_id).clone();
    let t: Vec<f64> = linspace(0., num_values as f64, num_values).collect();
    let mut samples: Vec<Vec<f32>> = vec!(vec!(); num_octaves);
    for j in 0..num_octaves {
        samples[j].extend(&wt[j * num_values..(j + 1) * num_values]);
    }
    Surface::new(samples).x(t.clone()).y(t.clone())
}

## Examining the basic wave shapes

Create a plot of the basic wave shapes (sine, triangle, ramp, square)

In [17]:
let wt_basic = wt_manager.get_table(basic_wave_id).unwrap();

// Prepare plot data
let num_tables = wt_basic.num_tables;
let num_samples = wt_basic.num_samples;
let mut plot = Plot::new();
for i in 0..num_tables {
    add_trace(&mut plot, wt_basic.get_wave(i), num_samples, i + 1);
}

// Create the plot
let layout = Layout::new()
    .grid(LayoutGrid::new().rows(2).columns(2).pattern(GridPattern::Independent).x_gap(0.15).y_gap(0.15),);
plot.set_layout(layout);
plot.notebook_display();

Take the tables and run an FFT on them, show the first 32 of 1024 harmonics.

In [18]:
let wt_basic = wt_manager.get_table(basic_wave_id).unwrap();
let harmonics = wt_basic.convert_to_harmonics();
let n = 32;
let mut plot = Plot::new();
for i in 0..num_tables {
    add_trace_complex(&mut plot, &harmonics[i], n, i + 1);
}

let layout = Layout::new()
    .grid(LayoutGrid::new().rows(2).columns(2).pattern(GridPattern::Independent).x_gap(0.15).y_gap(0.15),)
    .height(600);
plot.set_layout(layout);
plot.notebook_display();

Creating 4 lists of harmonics with 2048 entries each for waves with 2048 samples


Add a surface plot of the bandlimited wave shapes

In [75]:
let wt_basic = wt_manager.get_table(basic_wave_id).unwrap();
let mut plot = Plot::new();
let table_id = 2;
let trace = get_surface_plot(&wt_basic, table_id);
plot.add_trace(trace);

let layout = Layout::new().height(800);
plot.set_layout(layout);
plot.notebook_display();

Error: cannot find value `wt_manager` in this scope

Error: cannot find value `basic_wave_id` in this scope

# Convert wave to FFT and back

Let's load a simple wave table file and analyze it

In [70]:
let reader = WtReader::new("/Users/ingo/Documents/Programming/src/rust/wavetable");
let wt = reader.read_file("AKWF_0001.wav", Some(600)).unwrap();
println!("Got wavetable with {} tables, for {} octaves, with {} samples and {} values per octave",
    wt.num_tables,
    wt.num_octaves,
    wt.num_samples,
    wt.num_values);
let mut plot = Plot::new();
add_trace(&mut plot, wt.get_wave(0), wt.num_samples, 1);
plot.notebook_display();

Got wavetable with 1 tables, for 1 octaves, with 600 samples and 601 values per octave


Then run an FFT to get the list of harmonics.

In [71]:
let harmonics = wt.convert_to_harmonics();
let n = 50;
let mut plot = Plot::new();
add_trace_complex(&mut plot, &harmonics[0], n, 1);
plot.notebook_display();

Creating 1 lists of harmonics with 600 entries each for waves with 600 samples


We have two ways of generating the waveform from the list of harmonics:

- Additive by stacking sine waves, which requires calculating magnitude and phase from the FFT result
- Running an inverse FFT

Currently Wavetable uses the inverse FFT.

In [73]:
let mut wt_new = Wavetable::new(1, 1, 600); // Reserve space for 1 waveshapes with 1 octave table
wt_new.insert_harmonics(&harmonics, 44100.0).unwrap();
let mut plot = Plot::new();
add_trace(&mut plot, wt_new.get_wave(0), 600, 1);
plot.notebook_display();

Inserting harmonics for 1 octaves
Inserting harmonics into table of length 601


That does look close to the original shape. Now let's look at a surface plot of the bandlimited version.

In [74]:
use std::sync::Arc;

let mut wt_new = Wavetable::new(1, 11, 600); // Reserve space for 1 waveshapes with 11 octave tables
wt_new.insert_harmonics(&harmonics, 44100.0).unwrap();
let mut plot = Plot::new();
let trace = get_surface_plot(&Arc::new(wt_new), 0);
plot.add_trace(trace);

let layout = Layout::new().height(800);
plot.set_layout(layout);
plot.notebook_display();

Inserting harmonics for 11 octaves
Inserting harmonics into table of length 6611


# Load a complex wavetable and bandlimit it

In [84]:
let reader = WtReader::new("/Users/ingo/Documents/Programming/src/rust/wavetable");
let result = reader.read_file("ESW Digital - Flavors.wav", Some(2048));
if let Ok(wt) = result {
    let wt = WtManager::bandlimit(wt, 11, 44100.0);
    let mut plot = Plot::new();
    let trace = get_surface_plot(&wt, 10);
    plot.add_trace(trace);
    let layout = Layout::new().height(800);
    plot.set_layout(layout);
    plot.notebook_display();
};


Creating 256 lists of harmonics with 2048 entries each for waves with 2048 samples
